In [1]:
# !pip install yfinance
import yfinance as yf

In [2]:
# !pip install transformers
import transformers

/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
import numpy as np
# import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [4]:
start = datetime.datetime(2008,8,8)
end = datetime.datetime(2014,7,1)
test_start = datetime.datetime(2014,7,2)
test_end = datetime.datetime(2016,7,1)

In [64]:
# Amazon
Amazon = yf.download("AMZN", start, end)

# Meta
Meta = yf.download("META", start, end)

# Alphabet
Alphabet = yf.download("GOOG", start, end)

# Microsoft
Microsoft = yf.download("MSFT", start, end)

# Apple
Apple = yf.download("AAPL", start, end)

# IBM
IBM = yf.download("IBM", start, end)

# Meta
NVIDIA = yf.download("NVDA", start, end)

# Alphabet
Tesla = yf.download("TSLA", start, end)

# Salesforce
Salesforce = yf.download("CRM", start, end)

# AMD
AMD = yf.download("AMD", start, end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [65]:
Amazon.reset_index(inplace=True)
Meta.reset_index(inplace=True)
Alphabet.reset_index(inplace=True)
Microsoft.reset_index(inplace=True)
Apple.reset_index(inplace=True)
IBM.reset_index(inplace=True)
NVIDIA.reset_index(inplace=True)
Tesla.reset_index(inplace=True)
Salesforce.reset_index(inplace=True)
AMD.reset_index(inplace=True)

In [66]:
Amazon["Company"]='Amazon'
Meta["Company"]='Meta'
Microsoft["Company"]='Microsoft'
Apple["Company"]='Apple'
Alphabet["Company"]='Alphabet'
IBM["Company"]='IBM'
NVIDIA["Company"]='NVIDIA'
Tesla["Company"]='Tesla'
Salesforce["Company"]='Salesforce'
AMD["Company"]='AMD'

In [46]:
type(Meta.Date)

pandas.core.series.Series

In [71]:
def prepare_data(data, days=30):
    X, y, date = [], [], []
    for i in range(len(data) - days):
        X.append(data['Close'].values[i:i+days])
        y.append(data['Close'].values[i+days])
        date.append(np.datetime_as_string(data['Date'].values[i+days], unit='D'))
    return np.array(X), np.array(y),date

def accuracy(truth,predictions):
    correct = 0
    total = 0
    for i in range(1,len(truth)):
        total += 1
        if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
            correct += 1
        else:
            print(truth[i-1],truth[i],predictions[i])
    print(f"accuracy{correct/total}")
    return correct/total

def predict_LR(stock_name,company_name):
    train = yf.download(stock_name, start, end)
    test = yf.download(stock_name, test_start, test_end)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    train["Company"]=company_name
    test["Company"]=company_name

    # Prepare the data
    X_train, y_train, date_train = prepare_data(train)
    X_test, y_test,date_test = prepare_data(test)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # stock_price_map = {key: value for key, value in zip(date_test, y_train)}
    stock_prediction_map = {key: value for key, value in zip(date_test, predictions)}
    

    # Evaluate the model
    # plt.figure(figsize=(10, 5))
    # plt.plot(y_test, label='True Prices')
    # plt.plot(predictions, label='Predicted Prices')
    # plt.legend()
    # plt.title('Stock Price Prediction')
    # plt.show()

    accuracy(y_test,predictions)
    return stock_prediction_map
        

In [76]:
company_map = {'Amazon': 'AMZN', 'Meta': 'META', 'Google': 'GOOGL', 'Microsoft': 'MSFT', 'Apple': 'AAPL', 'IBM': 'IBM', 'NVIDIA': 'NVDA', 'Tesla': 'TSLA', 'Salesforce': 'CRM', 'AMD': 'AMD'}
# predict_LR("META","META")
# Switch keys and values
stock_prediction_map = {}
for key, value in company_map.items():
    stock_prediction_map[key] = predict_LR(value,key)
stock_prediction_map

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1])

16.726499557495117 16.756500244140625 16.632340536039983
16.756500244140625 16.788999557495117 16.752565522926258
16.64550018310547 16.579500198364258 16.68075910176626
16.579500198364258 16.701000213623047 16.5684819377262
17.09149932861328 17.159000396728516 17.00935494203441
16.95199966430664 17.118999481201172 16.924808204965252
17.118999481201172 16.950000762939453 17.147948378558297
16.950000762939453 17.297500610351562 16.9399017756529
17.117000579833984 16.487499237060547 17.155767101774828
16.566499710083008 16.525999069213867 16.628340782185315
16.559499740600586 16.194499969482422 16.593787338512897
16.160499572753906 16.090999603271484 16.189342600101448
16.121999740600586 15.873000144958496 16.143976720964012
15.873000144958496 15.920499801635742 15.844410484347126
16.110000610351562 15.848999977111816 16.114770449427112
16.135000228881836 15.768500328063965 16.148547392814976
15.569499969482422 15.322500228881836 15.62052603911912
15.322500228881836 15.415499687194824 15.

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953

24.4950008392334 24.790000915527344 24.4903007722498
25.13249969482422 25.142499923706055 25.122483637678542
25.142499923706055 25.145000457763672 25.131358452325205
25.145000457763672 25.329999923706055 25.084845719394345
25.22249984741211 25.532499313354492 25.198314840138817
25.5625 25.625 25.539443022119556
25.825000762939453 24.735000610351562 25.872574231407953
24.74250030517578 24.59000015258789 24.82835251198404
24.497499465942383 25.25 24.487562516912345
25.415000915527344 25.407499313354492 25.473737241141208
25.407499313354492 25.21500015258789 25.54293670762682
25.21500015258789 25.395000457763672 25.173926314802873
25.447500228881836 25.239999771118164 25.497054044540672
25.239999771118164 25.264999389648438 25.095687039465748
25.264999389648438 25.65999984741211 25.246874231005215
25.65999984741211 25.4375 25.77758544530061
24.467500686645508 25.1875 24.44712517586496
25.1875 25.02750015258789 25.286346780553075
25.02750015258789 25.1875 25.02570068032862
24.9050006866455


[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_2739/3608293953.py:14: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):


53.630001068115234 54.380001068115234 53.618384304575855
54.380001068115234 54.959999084472656 54.23212506117015
54.959999084472656 55.38999938964844 54.70789614213599
55.709999084472656 59.79999923706055 55.67143461470623
59.22999954223633 59.63999938964844 59.17844459313899
58.290000915527344 59.09000015258789 58.15326416600699
59.849998474121094 59.029998779296875 59.93157448230767
59.029998779296875 59.08000183105469 58.951209620217924
59.08000183105469 59.810001373291016 58.950108310965874
60.36000061035156 59.65999984741211 60.45827395471022
59.65999984741211 60.70000076293945 59.541973070772514
60.70000076293945 61.209999084472656 60.57354459276017
61.209999084472656 59.25 61.435280466858345
57.56999969482422 57.61000061035156 57.28911661171507
57.61000061035156 57.45000076293945 57.86352679672949
57.65999984741211 55.91999816894531 57.801700355609
56.560001373291016 57.22999954223633 56.39426510413114
58.459999084472656 58.29999923706055 58.68996556481986
56.90999984741211 57.9

{'Amazon': {'2014-08-14': 16.334515104582916,
  '2014-08-15': 16.707203063108047,
  '2014-08-18': 16.735310983114427,
  '2014-08-19': 16.632340536039983,
  '2014-08-20': 16.752565522926258,
  '2014-08-21': 16.768582774902058,
  '2014-08-22': 16.68075910176626,
  '2014-08-25': 16.5684819377262,
  '2014-08-26': 16.75143291079177,
  '2014-08-27': 17.00935494203441,
  '2014-08-28': 17.12282637593455,
  '2014-08-29': 16.893451033173555,
  '2014-09-02': 16.924808204965252,
  '2014-09-03': 17.147948378558297,
  '2014-09-04': 16.9399017756529,
  '2014-09-05': 17.328930252844252,
  '2014-09-08': 17.279077807878217,
  '2014-09-09': 17.155767101774828,
  '2014-09-10': 16.496453653678508,
  '2014-09-11': 16.628340782185315,
  '2014-09-12': 16.53520700845971,
  '2014-09-15': 16.593787338512897,
  '2014-09-16': 16.21212054614246,
  '2014-09-17': 16.38024525733426,
  '2014-09-18': 16.210961241892175,
  '2014-09-19': 16.32433174407053,
  '2014-09-22': 16.541568852517464,
  '2014-09-23': 16.20950328724

In [60]:
stock_prediction_map["Amazon"].keys()

dict_keys([numpy.datetime64('2014-08-14T00:00:00.000000000'), numpy.datetime64('2014-08-15T00:00:00.000000000'), numpy.datetime64('2014-08-18T00:00:00.000000000'), numpy.datetime64('2014-08-19T00:00:00.000000000'), numpy.datetime64('2014-08-20T00:00:00.000000000'), numpy.datetime64('2014-08-21T00:00:00.000000000'), numpy.datetime64('2014-08-22T00:00:00.000000000'), numpy.datetime64('2014-08-25T00:00:00.000000000'), numpy.datetime64('2014-08-26T00:00:00.000000000'), numpy.datetime64('2014-08-27T00:00:00.000000000'), numpy.datetime64('2014-08-28T00:00:00.000000000'), numpy.datetime64('2014-08-29T00:00:00.000000000'), numpy.datetime64('2014-09-02T00:00:00.000000000'), numpy.datetime64('2014-09-03T00:00:00.000000000'), numpy.datetime64('2014-09-04T00:00:00.000000000'), numpy.datetime64('2014-09-05T00:00:00.000000000'), numpy.datetime64('2014-09-08T00:00:00.000000000'), numpy.datetime64('2014-09-09T00:00:00.000000000'), numpy.datetime64('2014-09-10T00:00:00.000000000'), numpy.datetime64('20

semantic analysis example

In [74]:
from transformers import BertTokenizer, BertModel, pipeline

import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example sentence
sentence = "The quick brown fox jumps over the lazy dog."

# Tokenize the input sentence and get token IDs
inputs = tokenizer(sentence, return_tensors='pt')

# Get the outputs from the model
with torch.no_grad():
    outputs = model(**inputs)

# The last hidden state of the [CLS] token (sentence-level representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
sentiment_analysis = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

sentences = [
    "I love this product! It's amazing.",
    "Meta"
]
score_map = {"5 stars":5,"4 stars":4, "3 stars":3, "2 stars":2, "1 star":1}
# Analyze the sentiment of each sentence
for sentence in sentences:
    result = sentiment_analysis(sentence)[0]
    print(result)
    print(f"Sentence: {sentence}")
    print(f"Sentiment: {result['label']}, Score: {result['score']*score_map[result['label']]}")

{'label': '5 stars', 'score': 0.9424458742141724}
Sentence: I love this product! It's amazing.
Sentiment: 5 stars, Score: 4.712229371070862
{'label': '3 stars', 'score': 0.25634676218032837}
Sentence: Meta
Sentiment: 3 stars, Score: 0.7690402865409851


cosine similarity example

In [23]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

def compute_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Compute token embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Mean pooling
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example texts
text1 = "Microsoft"
text2 = "Musharraf to be impeached."

# Compute embeddings for both texts
embedding1 = compute_embeddings(text1)
embedding2 = compute_embeddings(text2)

# Convert embeddings to numpy arrays
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(similarity)

[[0.00980911]]


In [36]:
def sentiment(title_lst,stock_name):
    res = []
    for title in title_lst: 
        if cosine_similarity(compute_embeddings(title.lower()), compute_embeddings(stock_name.lower())) > 0.2 or stock_name.lower() in title.lower():
            result = sentiment_analysis(sentence)[0]
            res.append( (int(result['label'][0]),result['score']))
        else:
            res.append((0,0.0))
    return res


In [39]:
#read news data into news_dict
import pandas as pd
csv_file_path = 'Combined_News_DJIA.csv'
df = pd.read_csv(csv_file_path)
df = df.drop(columns=['Label'])
news_dict = df.set_index('Date').T.to_dict('list')
news_dict

Index(['Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25'],
      dtype='object')


In [31]:
#save customized data for training
columns = ['date', 'company', 'stock price', 'sentiment result', 'prediction']
customized_data = pd.DataFrame(columns=columns)

<class 'pandas.core.frame.DataFrame'>


In [87]:
news_dict

{'2008-08-08': ['b"Georgia \'downs two Russian warplanes\' as countries move to brink of war"',
  "b'BREAKING: Musharraf to be impeached.'",
  "b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'",
  "b'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'",
  'b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing"',
  "b'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.'",
  'b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side"',
  'b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it."',
  "b'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'",
  "

In [96]:
company_map = {
    "Amazon": Amazon,
    "Meta": Meta,
    "Alphabet": Alphabet,
    "Microsoft": Microsoft,
    "Apple": Apple,
    "IBM": IBM,
    "NVIDIA": NVIDIA,
    "Tesla": Tesla,
    "Salesforce": Salesforce,
    "AMD": AMD
}
# Display the dictionary
print(company_map)
for k,v in company_map.items():
    company_name = k
    datafram = v
# np.datetime_as_string(data['Date'].values[i+days], unit='D')
    print(type(row['Date']))
    print(row['Date'].strftime('%Y-%m-%d'))
    for index, row in datafram.iterrows():
        print(f"Date: {row['Date']}")
        print(f"Company: {row['Company']}")
        new_row = {
        'date': row['Date'],
        'company': company_name,
        'stock price': row['Close'],
        'sentiment result': sentiment(news_dict[row['Date'].strftime('%Y-%m-%d')],company_name),
        'prediction':stock_prediction_map[company_name][row['Date'].strftime('%Y-%m-%d')]
        }
        customized_data = customized_data.append(new_row, ignore_index=True)
customized_data

{'Amazon':            Date       Open       High        Low    Close  Adj Close  \
0    2008-08-08   3.839000   4.060500   3.814500   4.0255     4.0255   
1    2008-08-11   4.009000   4.587500   3.989000   4.4045     4.4045   
2    2008-08-12   4.366000   4.424000   4.305000   4.3625     4.3625   
3    2008-08-13   4.314000   4.412500   4.227000   4.3345     4.3345   
4    2008-08-14   4.285500   4.437500   4.261000   4.4015     4.4015   
...         ...        ...        ...        ...      ...        ...   
1478 2014-06-24  16.377001  16.486500  16.136999  16.2080    16.2080   
1479 2014-06-25  16.216499  16.407499  16.087000  16.3720    16.3720   
1480 2014-06-26  16.424500  16.432501  16.106501  16.2845    16.2845   
1481 2014-06-27  16.294001  16.326000  16.163500  16.2285    16.2285   
1482 2014-06-30  16.249001  16.349001  16.112000  16.2390    16.2390   

         Volume Company  
0     183254000  Amazon  
1     501404000  Amazon  
2     160530000  Amazon  
3     144176000  Ama